In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import fastf1
from src.plotset import setup_plot
from fastf1 import plotting

setup_plot()

In [ ]:
fastf1.Cache.enable_cache('./f1_cache')
fastf1.Cache.get_cache_info()

In [ ]:
session = fastf1.get_session(2025,14,'FP1')
session.load()

In [ ]:
drivers = session.drivers

In [ ]:
tire = 'MEDIUM'
drivers = session.laps.pick_compounds(tire).Driver.unique()

In [ ]:
speed_dict = {}

for driver in drivers:
    dabb = session.get_driver(driver).Abbreviation
    color = plotting.get_driver_color(identifier=dabb,session=session)
    df = session.laps.pick_drivers(dabb).pick_quicklaps().pick_compounds(tire)
    min_speed, max_speed = [], []
    for i, lap in df.iterlaps():
        min_speed.append(lap.get_car_data().Speed.min())
        max_speed.append(lap.get_car_data().Speed.max())   
    speed_dict[session.get_driver(dabb).Abbreviation] = [max(min_speed), max(max_speed), color]

In [ ]:
speed_df = pd.DataFrame(speed_dict).T.set_axis(['min_speed','max_speed','color'],axis=1)

In [ ]:
# fig, ax = plt.subplots(figsize=(12,6))
# sns.scatterplot(data=speed_df, x='min_speed', y='max_speed',hue=speed_df.index,palette=speed_df.color.to_list(),legend=False,s=1000,ax=ax)

In [ ]:
# --- 1️⃣ Define ceilings (fastest driver speeds) ---
min_speed_ceiling = speed_df['min_speed'].max()
max_speed_ceiling = speed_df['max_speed'].max()

# --- 2️⃣ Calculate negative gaps (driver deficit) ---
speed_df['min_gap'] = speed_df['min_speed'] - min_speed_ceiling  # ≤ 0
speed_df['max_gap'] = speed_df['max_speed'] - max_speed_ceiling  # ≤ 0

# --- 3️⃣ Sort dataframe based on gaps (best performer first) ---
speed_df_sorted = speed_df.sort_values(by='min_gap', ascending=False)

# ----4️⃣ Sort again for max_gap (might differ from min_gap order) ----
speed_df_sorted2 = speed_df.sort_values(by='max_gap', ascending=False)

# Top driver in slow speeds
best_driver = speed_df_sorted.index[0]
best_speed = speed_df_sorted['min_speed'].iloc[0]
best_color = speed_df_sorted['color'].iloc[0]

# Top driver in straights
best_driver2 = speed_df_sorted2.index[0]
best_speed2 = speed_df_sorted2['max_speed'].iloc[0]
best_color2 = speed_df_sorted2['color'].iloc[0]

In [ ]:
# --- Plotting ---

setup_plot(xyticksize=20,axeslabel=20)

fig, axes = plt.subplots(2, 1, figsize=(16, 10))

# ---- Plot 1: Slow corner speed gaps ----
axes[0].bar(speed_df_sorted.index,
            speed_df_sorted['min_gap'],
            color=speed_df_sorted['color'])
axes[0].set_ylabel('Δ Speed (km/h)')
axes[0].set_title('Gap to Best Slow Corner Speed')
axes[0].axhline(0, color='black', linewidth=10)

# ---- Plot 2: Straight-line speed gaps ----
axes[1].bar(speed_df_sorted2.index,
            speed_df_sorted2['max_gap'],
            color=speed_df_sorted2['color'])
axes[1].set_ylabel('Δ Speed (km/h)')
axes[1].set_title('Gap to Best Straight-line Speed')
axes[1].axhline(0, color='black', linewidth=10)

axes[0].set_xlim(-0.5, len(speed_df_sorted) - 0.5)
axes[1].set_xlim(-0.5, len(speed_df_sorted2) - 0.5)

axes[0].text(
    x=0, y=-1.2,                           # position (x = bar index, y = just below axhline)
    s=f"{best_speed:.0f}\nkm/h",           # text to display
    color=best_color, fontsize=18,
    ha='center', va='bottom', fontweight='bold'
)

axes[1].text(
    x=0, y=-1.7,                           # position (x = bar index, y = just below axhline)
    s=f"{best_speed2:.0f}\nkm/h",           # text to display
    color=best_color2, fontsize=18,
    ha='center', va='bottom', fontweight='bold'
)

plt.tight_layout()
plt.show()